In [6]:
from websockets.client import connect
import json
import os
import dotenv

from src.data_ingestion.listen_news import send_new

dotenv.load_dotenv()

auth_msg = {
  "action": "auth",
  "key": os.environ["APCA_API_KEY_ID"],
  "secret": os.environ["APCA_API_SECRET_KEY"]
}
start_listen_msg= {
      "action": "listen",
      "data": {
        "streams": ["trade_updates"]
      }
    }
async with connect("wss://paper-api.alpaca.markets/stream") as websocket:
    await websocket.send(json.dumps(auth_msg))
    await websocket.send(json.dumps(start_listen_msg))
    data = await websocket.recv()

    async for message in websocket:
      print(data)
      send_new(json.loads(data))


b'{"stream":"authorization","data":{"action":"authenticate","status":"authorized"}}'


In [ ]:
auth

{'action': 'auth',
 'key': 'PKVUS0LW2MYNR51ID5ZS',
 'secret': 'AUuz3zFpHVisDsfhUKeBkMUeUEXOjOXX9BRSUilE'}

In [ ]:
import json
from websocket import create_connection
import websocket 

m = {
  "action": "listen",
  "data": {
    "streams": ["trade_updates"]
  }
}

websocket.enableTrace(True)
ws = create_connection("wss://paper-api.alpaca.markets/stream", extra_headers=auth)
ws.send(json.dumps(auth))
ws.send(json.dumps(m))

result = ws.recv()
print('Result: {}'.format(result))

--- request header ---
GET /stream HTTP/1.1
Upgrade: websocket
Host: paper-api.alpaca.markets
Origin: https://paper-api.alpaca.markets
Sec-WebSocket-Key: N/jCdOoHTW65rlQ6IB0wuQ==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Tue, 16 Apr 2024 01:01:38 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: v3xXd2EPS/nf+1+/q5Q6UsS0MnQ=
Strict-Transport-Security: max-age=31536000; includeSubDomains
X-Request-ID: 9901c1b40e2129466b70e094e3340e5a
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET, OPTIONS, PUT, DELETE, POST, PATCH
Access-Control-Allow-Headers: Apca-Api-Key-Id, Apca-Api-Secret-Key, Authorization
Access-Control-Max-Age: 1728000
-----------------------
++Sent raw: b'\x81\xe7<GXzGe9\x19H.7\x14\x1e}xX]2,\x12\x1ekxXW"!X\x06gz*w\x11\r)\x0c\x0b\x0fHq\x1e\x16(\tv\x11>\t\x1d\x0bX\x10gz\tY$*\x1fHebZ\x1e\x06\r\x0fFt"<L\x0f\x0e\x13O\x03+\

Result: b'{"stream":"authorization","data":{"action":"authenticate","status":"authorized"}}'


In [ ]:
ws.recv()

++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80\xbe\x10\xe6x'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80\xb1\xd33\xa3'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80\xd5\x8c\x84\r'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80\xe9\xdfc\xa5'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80T\x06^\xbe'
++Sent decoded: fin=1 opcode=10 data=b''


KeyboardInterrupt: 

## Get last news and send to Kafka topic

In [ ]:
import pandas as pd
import requests
import json 


url = "https://data.alpaca.markets/v1beta1/news"#?symbols=COIN,BTCUSD"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKVUS0LW2MYNR51ID5ZS",
    "APCA-API-SECRET-KEY": "AUuz3zFpHVisDsfhUKeBkMUeUEXOjOXX9BRSUilE"
}

response = requests.get(url, headers=headers)

news_response = json.loads(response.text)

In [ ]:
news_response_df = pd.DataFrame(news_response["news"])
news_response_df

In [ ]:
from src.data_ingestion.listen_news import send_new

for i in news_response_df.to_dict(orient='records'):
    send_new(i)